In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-darkgrid")

# -----------------------------
# Load input data
# -----------------------------
df_fwd = pd.read_excel("A.xlsx", sheet_name="NBP Forwards", parse_dates=["Date"])
df_da  = pd.read_excel("A.xlsx", sheet_name="NBP Outturn",  parse_dates=["Date"])

# -----------------------------
# Reporting Months
# -----------------------------
reporting_months = pd.date_range("2024-04-01", "2026-03-01", freq="MS").to_list()
first_rm = reporting_months[0]

# -----------------------------
# Seasons metadata
# -----------------------------
SEASONS = {
    "S24": {"exposure":100_000,"starting_volume":80_000,"demand":100_000,"starting_wap":156.0,
            "horizon":pd.Timestamp("2024-03-01"),"delivery_end":pd.Timestamp("2024-09-01"),"optimise":False},
    "W24": {"exposure":150_000,"starting_volume":108_000,"demand":150_000,"starting_wap":139.0,
            "horizon":pd.Timestamp("2024-09-01"),"delivery_end":pd.Timestamp("2025-03-01"),"optimise":True},
    "S25": {"exposure":100_000,"starting_volume":39_000,"demand":100_000,"starting_wap":101.0,
            "horizon":pd.Timestamp("2025-03-01"),"delivery_end":pd.Timestamp("2025-09-01"),"optimise":True},
    "W25": {"exposure":150_000,"starting_volume":9_000,"demand":150_000,"starting_wap":106.0,
            "horizon":pd.Timestamp("2025-09-01"),"delivery_end":pd.Timestamp("2026-03-01"),"optimise":True},
}


In [2]:
SPREAD_MAP = {
    "W24": ("W24", "S25"),
    "S25": ("S25", "W25"),
    "W25": ("W25", "S25")   # fallback spread
}

def compute_spread_series(season):
    leg1, leg2 = SPREAD_MAP[season]
    s = df_fwd[["Date", leg1, leg2]].dropna().copy()
    s["Spread"] = s[leg1] - s[leg2]
    return s


In [3]:
def compute_wap(df, season, starting_wap):
    waps = []
    wap = starting_wap
    prev_vol = df["HedgeVolume"].iloc[0]

    for i, r in df.iterrows():
        vol, inc = r["HedgeVolume"], r["Increment"]

        if pd.isna(vol):
            waps.append(np.nan)
            continue

        if i == 0:
            waps.append(wap)
            continue

        if inc > 0:
            px = df_fwd.loc[df_fwd["Date"] <= r["ReportingMonth"], season]
            exec_px = px.iloc[-1] if not px.empty else df_fwd[season].dropna().iloc[0]
            wap = (wap * prev_vol + exec_px * inc) / vol

        waps.append(wap)
        prev_vol = vol

    df["WAP"] = waps
    return df


def compute_mtm(df, season, horizon, delivery_end):
    out = []
    for _, r in df.iterrows():
        rm, vol, wap = r["ReportingMonth"], r["HedgeVolume"], r["WAP"]

        if pd.isna(vol) or pd.isna(wap):
            out.append(np.nan)
            continue
        if rm == first_rm:
            out.append(np.nan)
            continue

        pm_start = (rm - pd.DateOffset(months=1)).replace(day=1)
        pm_end   = rm.replace(day=1) - pd.Timedelta(days=1)

        if rm <= horizon:
            w = df_fwd.loc[(df_fwd["Date"]>=pm_start)&(df_fwd["Date"]<=pm_end), season]
        elif rm <= delivery_end:
            w = df_da.loc[(df_da["Date"]>=pm_start)&(df_da["Date"]<=pm_end), "DA"]
        else:
            out.append(np.nan)
            continue

        if w.empty:
            out.append(np.nan)
        else:
            px = w.mean()
            out.append((wap - px) * vol / 100)

    df["MtM"] = out
    return df


In [4]:
def run_spread_scenario(season, base_params, window, sig_buy, sig_sell, hedge_target):

    horizon      = base_params["horizon"]
    delivery_end = base_params["delivery_end"]
    start_vol    = base_params["starting_volume"]
    target_vol   = hedge_target * base_params["demand"]
    start_wap    = base_params["starting_wap"]

    spread_df = compute_spread_series(season)
    spread_df["MA"] = spread_df["Spread"].rolling(window).mean()
    spread_df["STD"] = spread_df["Spread"].rolling(window).std()
    spread_df = spread_df.bfill()

    rows = []
    vol = start_vol

    for rm in reporting_months:

        if rm == first_rm:
            rows.append([rm, vol, 0.0])
            continue

        if rm <= horizon:
            sub = spread_df.loc[spread_df["Date"]<=rm].iloc[-1]
            z = (sub["Spread"] - sub["MA"]) / sub["STD"] if sub["STD"]>0 else 0

            base_inc = (target_vol - start_vol) / max(1,len([m for m in reporting_months if first_rm<m<=horizon]))

            if z < -sig_buy:
                inc = base_inc * 1.5
            elif z > sig_sell:
                inc = base_inc * 0.5
            else:
                inc = base_inc

            inc = max(0, min(inc, target_vol - vol))
            vol += inc
            rows.append([rm, vol, inc])
            continue

        if rm <= delivery_end:
            vol = target_vol
            rows.append([rm, vol, 0.0])
            continue

        rows.append([rm, np.nan, np.nan])

    df = pd.DataFrame(rows, columns=["ReportingMonth","HedgeVolume","Increment"])
    df = compute_wap(df, season, start_wap)
    df = compute_mtm(df, season, horizon, delivery_end)
    return df


In [15]:
def score_spread(df, demand, hedge_target):
    mtm = df["MtM"].dropna()
    if len(mtm)<3: return -1e12

    pnl = mtm.sum()
    vol_penalty = mtm.std()

    final_vol = df["HedgeVolume"].dropna().iloc[-1]
    ratio = final_vol / demand
    ratio_penalty = abs(ratio - hedge_target)

    if ratio < 0.5:
        return -1e9

    return 0.4*pnl - 1.0*vol_penalty - 1*ratio_penalty


WINDOWS = [20,30,45]
SIGMA_BUY  = [1,2,3]
SIGMA_SELL = [1,2,3]
HEDGE_TARGETS = [0.50, 0.60, 0.70, 0.80, 0.90, 1.0]   

def run_spread_global(param):
    total = 0
    res = {}
    for s in ["W24","S25","W25"]:
        p = SEASONS[s]
        hed = run_spread_scenario(s,p,
                                  window=param["window"],
                                  sig_buy=param["sig_buy"],
                                  sig_sell=param["sig_sell"],
                                  hedge_target=param["hedge_target"])
        total += score_spread(hed, p["demand"], param["hedge_target"])
        res[s]=hed
    return res, total


def optimise_spread():
    best = {"score":-1e20,"params":None,"results":None}
    hist=[]
    for w in WINDOWS:
        for sb in SIGMA_BUY:
            for ss in SIGMA_SELL:
                for ht in HEDGE_TARGETS:

                    ps={"window":w,"sig_buy":sb,"sig_sell":ss,"hedge_target":ht}

                    results,score = run_spread_global(ps)
                    hist.append({"params":ps,"score":score})

                    if score>best["score"]:
                        best["score"]=score
                        best["params"]=ps
                        best["results"]=results
    return best,hist


In [16]:
best_spread, spread_history = optimise_spread()
print("BEST PARAMETERS:")
best_spread["params"]


BEST PARAMETERS:


{'window': 45, 'sig_buy': 1, 'sig_sell': 3, 'hedge_target': 1.0}

In [17]:
OUTPUT_FILE = "Spread_Optimised_Hedge_Output.xlsx"
FORMATTED_FILE = "Spread_Optimised_Hedge_Formatted.xlsx"
